In [2]:
import features
import lizard
from git.repo.base import Repo
import glob
import os
import pandas as pd
from pathlib import Path
import requests
import shutil
import stat
import urllib.request
import yaml

In [4]:
url = 'https://github.com/lucidrains/DALLE2-pytorch'

[user_name, repo_name] = url.rsplit('/', 2)[1:]
working_dir = Path(os.getcwd())
temp_dir = working_dir / "tmp" / user_name / repo_name
Repo.clone_from(url, temp_dir)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v https://github.com/lucidrains/DALLE2-pytorch C:\Programming\College\software_design\Cyclomatic-Complexity-Analyzer\tmp\lucidrains\DALLE2-pytorch
  stderr: 'fatal: destination path 'C:\Programming\College\software_design\Cyclomatic-Complexity-Analyzer\tmp\lucidrains\DALLE2-pytorch' already exists and is not an empty directory.
'

In [5]:
def flatten_nested_functions(funcs: list[features.Function]):
    i = 0
    while i < len(funcs):
        func = funcs[i]
        for nested_func in func.nested_funcs:
            nested_func.name = f"{func.name}.{nested_func.name}"
            funcs.append(nested_func)
        func.nested_funcs.clear()
        i += 1

file_name_prefix_len = len(str(temp_dir))

files = glob.glob(str(temp_dir / "**" / "*.py"))
outp = dict()
files_data = []
for file in files:
    name = file.split('/')[-1][:-3]
    if name == '__init__':
        continue
    lizard_analysis = lizard.analyze_file(file)
    extra_analysis: features.SourceFile = features.analyze_file(file)
    extra_functions = extra_analysis.functions
    flatten_nested_functions(extra_functions)
    extra_analysis: dict[tuple[str, int], features.Function] = {(func.name, func.start_line): func for func in extra_functions}
    functions = []
    for func in lizard_analysis.function_list:
        key = (func.name, func.start_line)
        extra = extra_analysis[key]
        functions.append({
            'name': func.name,
            'start_line': func.start_line,
            'nloc': func.nloc,
            'CCN': func.cyclomatic_complexity,
            'enclosing_class': extra.enclosing_class,
            'max_depth': extra.max_depth,
            'branches': extra.branches,
            'calls': extra.calls,
            'returns': extra.returns,
            'raises': extra.raises,
            'assertions': extra.assertions,
        })

    df = pd.DataFrame(data=functions, columns=['name', 'start_line', 'nloc', 'CCN', 'enclosing_class', 'max_depth', 'branches', 'calls', 'returns', 'raises', 'assertions'])
    pretty_file_name = file[len(str(temp_dir)):]
    outp[file[file_name_prefix_len:]] = df
    if '\\' in pretty_file_name:
        [file_dir, file_name] = pretty_file_name.rsplit('\\', 1)
    else:
        [file_dir, file_name] = pretty_file_name.rsplit('/', 1)
    files_data.append({
        'file_dir': file_dir,
        'file_name': file_name,
        'nloc': lizard_analysis.nloc,
        'CCN': lizard_analysis.CCN,
        'func_token': lizard_analysis.token_count,
    })
files_df = pd.DataFrame(data=files_data, columns=['file_dir', 'file_name', 'nloc', 'CCN', 'func_token'])


In [6]:
for (file, funcs) in outp.items():
    funcs = funcs[funcs.CCN > 1]
    if not funcs.empty:
        print(f"{file}:\n{funcs.sort_values(by=['CCN', 'nloc'], ascending=[False, False])}\n")

\dalle2_pytorch\cli.py:
      name  start_line  nloc  CCN enclosing_class  max_depth  branches  calls  \
0  safeget           9     2    2            None          0         0      2   

   returns  raises  assertions  
0        1       0           0  

\dalle2_pytorch\dalle2_pytorch.py:
                          name  start_line  nloc  CCN    enclosing_class  \
138                   __init__        1840   168   29               Unet   
145                   __init__        2459   154   27            Decoder   
141                    forward        2150   124   27               Unet   
155         p_sample_loop_ddim        2891    76   18            Decoder   
158                     sample        3108    72   16            Decoder   
..                         ...         ...   ...  ...                ...   
48                       clear         331     4    2  OpenAIClipAdapter   
58                       clear         402     4    2    OpenClipAdapter   
85             p2_reweigh_l

In [7]:
print(files_df)

           file_dir             file_name  nloc  CCN  func_token
0   \dalle2_pytorch                cli.py    40    6         326
1   \dalle2_pytorch     dalle2_pytorch.py  2284  479       19142
2   \dalle2_pytorch          optimizer.py    28    7         189
3   \dalle2_pytorch          tokenizer.py   152   53        1370
4   \dalle2_pytorch           trackers.py   479  131        3436
5   \dalle2_pytorch            trainer.py   511  137        4235
6   \dalle2_pytorch      train_configs.py   306   45        1928
7   \dalle2_pytorch              utils.py    21    8         119
8   \dalle2_pytorch            version.py     1    0           3
9   \dalle2_pytorch          vqgan_vae.py   557  105        4312
10  \dalle2_pytorch  vqgan_vae_trainer.py   189   29        1425
11  \dalle2_pytorch           __init__.py     6    0          46


In [7]:
def remove_readonly(f, path, _):
    os.chmod(path, stat.S_IWRITE)
    f(path)

shutil.rmtree(working_dir / "tmp", onerror=remove_readonly)